In [ ]:
!pip install scanpy

In [2]:
import numpy as np
import pandas as pd
import sys
import scanpy as sc
import os,gc
import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Processing

1. log transform / arcsine transform

2. averaging intensity by subtracting each cell's mean intensity from it

3. normalization by each cell

In [4]:
filedir = '/content/drive/MyDrive/Colab Notebooks/Living Cell imaging/Data'
h5ad_dir = os.path.join(filedir,"xy12.h5ad")
adata = sc.read_h5ad(h5ad_dir)

In [12]:
adata.X

array([[0.49854719, 0.48689214, 0.50971685, ..., 0.57934206, 0.6109046 ,
        0.60800616],
       [0.47939314, 0.48779239, 0.46786861, ..., 0.90491286, 0.85663161,
        0.        ],
       [0.75777552, 0.77580687, 0.88184772, ..., 0.76486667, 0.79749776,
        0.78302265],
       ...,
       [0.        , 0.        , 0.        , ..., 0.59687059, 0.60904625,
        0.59778016],
       [0.        , 0.        , 0.        , ..., 0.73593582, 0.68335322,
        0.66272857],
       [0.        , 0.        , 0.        , ..., 0.72133564, 0.73016454,
        0.75075764]])

In [11]:
adata.X[np.isnan(adata.X)] = 0

In [15]:
value = adata.X.flatten()
len(np.where(value == 0)[0])

221800

In [ ]:
#log1p
adata.layers["log_transformed"] = np.log1p(adata.X)
#normailize
adata.layers["ave_inten"] = adata.layers["log_transformed"] - np.nanmean(adata.layers["log_transformed"],axis=1,keepdims=True)
adata.X = adata.layers["ave_inten"]/np.sqrt(np.nanvar(adata.layers["ave_inten"],axis=1,keepdims=True))


In [ ]:
#build a temp anndata for test (only include one well)
filedir = '/content/drive/MyDrive/Colab Notebooks/Living Cell imaging/Data'
h5ad_dir = os.path.join(filedir,"processed_xy12.h5ad")
adata.write(h5ad_dir)

In [ ]:
#2 after ave
plt.figure(figsize=(100,20))

for cell in adata.layers["ave_inten"]:
    plt.plot(cell)

plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Time Series of Each Cell')
plt.show()

#2 log
plt.figure(figsize=(100,20))

for cell in adata.layers["log_transformed"]:
    plt.plot(cell)

plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Time Series of Each Cell')
plt.show()